In [1]:
import pandas as pd
import torch
import numpy as np

## Part 1: Setup

In [2]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")

In [3]:
#Display cell to practice slicing columns
#test_df.iloc[:,0:-1] #X values 
#test_df.iloc[:,-1].values #Y values

In [4]:
test_ds = torch.utils.data.TensorDataset(
    torch.tensor(test_df.iloc[:,0:-1].values, dtype=torch.float64), # X Values
    torch.tensor(test_df.iloc[:,-1].values, dtype=torch.float64), # Y Values
)

train_ds = torch.utils.data.TensorDataset(
    torch.tensor(train_df.iloc[:,0:-1].values, dtype = torch.float64), # X Values
    torch.tensor(train_df.iloc[:,-1].values, dtype = torch.float64), # Y Values
)

In [5]:
#Create the datasets for testing and training
testX, testY = test_ds[:]
testY = testY.view(-1,1)
trainX, trainY = train_ds[:]
trainY = trainY.view(-1,1)

In [6]:
#trainY Display Code to check for correctness

In [7]:
#testY[:5] Display Code to check for correctness

In [8]:
#Q1 

#How many bytes does trainX consume

#Consumption (Bytes) = num of elements * element size
trainX_elem_size = trainX.element_size()
trainX_num_elements = trainX.numel()
consumption = trainX_elem_size * trainX_num_elements
consumption

#print(f"The tensor trainX takes up {consumption} kilobytes")

83520

In [9]:
#Q2

#Convert trainX to float16
trainX_flt16 = trainX.to(dtype=torch.float16)

#Then convert back again to float64
trainX_flt64 = trainX_flt16.to(dtype=torch.float64)

#Take the resulting tensor (trainX_flt64) and subtract from the original (floatX)
difference_tensor = trainX - trainX_flt64

#Make the elements in diff tensor absolute
abs_diff_tensor = torch.abs(difference_tensor)
#Find the max value in the tensor
max_abs_diff_tensor = torch.max(abs_diff_tensor)

#Display as a python float
max_abs_diff_tensor.item()

0.0

In [10]:
#Q3 

#Is a CUDA GPU available on your VM?
torch.cuda.is_available()

False

## Part 2: Prediction with Hardcoded Model

In [11]:
#Encode these assumptions as coefficients in a tensor by pasting the following:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [12]:
#Q4 

#What is the predicted number of deaths for the first census tract?
first_census_tract = (testX[0] @ coef).item()
first_census_tract

9.844

In [13]:
#Q5 

#What is the average number of predicted deaths, over the whole testX dataset?
pred_deaths = testX @ coef #calculate pred_death tensor
average_deaths = pred_deaths.mean().item() #Find average of pred_death tensor

average_deaths

12.073632183908048

## Part 3

Say y = x^2 - 8x + 19 find the value x that minimizes y,

In [14]:
#Q6 
#first, what is y when x is a tensor containing 0.0

x = torch.tensor(0.0)
y = x ** 2 - 8 * x + 19
float(y)

19.0

In [15]:
#Q7 

# What x value minimizes y?
x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr = .1)

for run in range(100):

    #Find value for y
    y = x ** 2 - 8 * x + 19

    #Find gradient value
    optimizer.zero_grad() #Reset the grad on optimizer
    y.backward()

    #Update the x via optimizer
    optimizer.step()

optimal_x = x.item()
optimal_x

3.999999523162842

## Part 4: Linear Regression

In [16]:
#Looking at the shape of trainX
trainX.shape

torch.Size([1044, 10])

In [17]:
#Looking at shape of trainY
trainY.shape

torch.Size([1044, 1])

In [18]:
#Q8 

#What is the MSE (mean-square-error) when we make predictions using the zero vector?
#Code from ml_opt.ipynb and adjusted for trainX and trainY shape
coef = torch.zeros((10,1), dtype=torch.float64, requires_grad=True)
loss_fn = torch.nn.MSELoss()
mse_zero_vector = loss_fn(trainX @ coef, trainY) #Compare trainX @ coef to trainY
mse_zero_vector.item()




197.8007662835249

### Optimization

In [19]:
#Setup a training dataset data loader like this:
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

#Use torch.optim.SGD and use 0.000002 learning rate
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD([coef], lr = 0.000002)

#Run for 500 epochs code from ml_opt.ipynb and adjusted for this exercise
for epoch in range(500):
    for batchX, batchY in dl: 
        predictions = batchX @ coef
        loss = loss_fn(predictions, batchY)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    #Debugger to see if MSE tensors are being outputted
    # X, Y = ds[:]
    # print(loss_fn(X @ coef, Y))
    


In [20]:
#Q9

# What is the MSE over the training data from the above training?
loss_fn(trainX @ coef, trainY).item()

26.622213540277947

In [21]:
#Q10 

#What is the MSE over the test data?
loss_fn(testX @ coef, testY).item()

28.832880030053065